In [1]:
import pandas as pd

In [2]:
pd.read_csv("data/all_platforms_classified.csv")

,platform,image,full_image_path,url,settings,category
0,facebook,initial_load_20251119T195333Z.png,/generaloutput/facebook/screenshots/sections/i...,https://accountscenter.facebook.com/password_a...,"[{'setting': 'Change password', 'description':...",security_authentication
1,facebook,after_click__Your_information_and_permissions_...,/generaloutput/facebook/screenshots/sections/a...,https://accountscenter.facebook.com/info_and_p...,"[{'setting': 'Export your information', 'descr...",data_collection_tracking
2,facebook,after_click__Ad_preferences_20251119T195345Z.png,/generaloutput/facebook/screenshots/sections/a...,https://accountscenter.facebook.com/ads,"[{'setting': 'Profiles', 'description': 'A nav...",data_collection_tracking
3,facebook,after_click__Personal_details_20251119T195346Z...,/generaloutput/facebook/screenshots/sections/a...,https://accountscenter.facebook.com/personal_info,"[{'setting': 'Contact info', 'description': 'M...",identity_personal_info
4,facebook,password_and_security_20251119T195347Z.png,/generaloutput/facebook/screenshots/sections/p...,https://accountscenter.facebook.com/personal_info,"[{'setting': 'Contact info', 'description': ""M...",identity_personal_info
5,facebook,after_click__Account_ownership_and_control_202...,/generaloutput/facebook/screenshots/sections/a...,https://accountscenter.facebook.com/personal_i...,"[{'setting': 'Memorialization', 'description':...",visibility_audience
6,facebook,personal_details_20251119T195413Z.png,/generaloutput/facebook/screenshots/sections/p...,https://accountscenter.facebook.com/personal_i...,"[{'setting': 'Contact info', 'description': 'M...",identity_personal_info
7,facebook,account_ownership_and_control_20251119T195441Z...,/generaloutput/facebook/screenshots/sections/a...,https://accountscenter.facebook.com/personal_info,"[{'setting': 'Contact info', 'description': 'M...",identity_personal_info
8,facebook,end_of_run_20251119T195457Z.png,/generaloutput/facebook/screenshots/sections/e...,https://accountscenter.facebook.com/personal_info,"[{'setting': 'Contact info', 'description': ""M...",identity_personal_info
9,googleaccount,initial_load_20251119T201600Z.png,/generaloutput/googleaccount/screenshots/secti...,https://myaccount.google.com/data-and-privacy,"[{'setting': 'Web & App Activity', 'descriptio...",data_collection_tracking
